In [2]:
import autograd.numpy as np
from autograd import grad

In [14]:
class NANDNeuron():
    def __init__(self, n):
        self.n = n
        self.epsilons = np.full((n), 0.0) #np.random.uniform(-0.2, 0.2, n)
        self.effector = -1
        
    def __repr__(self):
        return str(self.epsilons)
        
    def __beforePresent__(self):
        self.effector = -1
        self.delta = 0
        self.grad = np.zeros(self.n)
    
    def setCheckGrad(self, g):
        self.checkGrad = g
        
    def setGrad(self, g):
        self.grad += g
        
    def getCheckGrad(self):
        return self.checkGrad
    
    def getGrad(self):
        return self.grad
    
    def getDelta(self):
        return self.delta
    
    def setDelta(self, delta):
        self.delta = delta
    
    def present(self, inputs):
        self.__beforePresent__()
        mus = np.maximum(self.epsilons, inputs)
        self.mus = mus
        self.inputs = inputs
        out = np.min(mus)
        
        # Determin which had effect
        for i in range(0, len(mus)):
            if out == mus[i]:
                self.effector = i
        
        self.output = -out
        return self.output

    def getInput(self):
        return self.inputs
    
    def getMus(self):
        return self.mus
    
    def getOutput(self):
        return self.output
    
    def updateWeights(self, grad):
        self.epsilons = self.epsilons - grad
#         for i in range(0, len(self.epsilons)):
#             if self.epsilons[i] > 1:
#                 self.epsilons[i] = 1
#             elif self.epsilons[i] < -1:
#                 self.epsilons[i] = -1
    
    def getWeights(self):
        return self.epsilons
    
    def getEffector(self):
        return self.effector

class NANDLayer():
    def __init__(self, inputs, nodes):
        self.layer = []
        
        for i in range(0, nodes):
            self.layer.append(NANDNeuron(inputs))
            
    def __repr__(self):
        s = ""
        for l in self.layer:
            s += (str(l) + " ,")
            
        return s

    def getLayer(self):
        return self.layer
    
    def setFolowingLayer(self, l):
        self.folowingLayer = l
        
    def present(self, inputs):
        out = []
        for n in self.layer:
            out.append(n.present(inputs))
        
        return out
    
    def backprop(self, prediction, target, output=False):
        for n in range(0, len(self.layer)):
            grad = None
            if output:
                grad = gradientOutputLayer(self.layer[n], target, prediction)
            else:
                grad = gradientHiddenLayer(self.layer[n], n, self.folowingLayer)
                
            self.layer[n].setGrad(grad)
            self.layer[n].updateWeights(grad * 1)
    
class NANDNetwork():
    def __init__(self, nIns, lParams, nOuts):
        self.layers = []
        
        lParams.append(nOuts)
        inputs = nIns
        for l in lParams:
            self.layers.append(NANDLayer(inputs, l))
            inputs = l
                    
        for i in range(1, len(self.layers)):
            self.layers[i-1].setFolowingLayer(self.layers[i])
        
    def __repr__(self):
        s = ""
        for l in range(0, len(self.layers)):
            s += ("Layer " + str(l+1) + " -> " + str(self.layers[l]) + "\n")
            
        return s
        
    def getLayers(self):
        return self.layers
        
        
    def fowardprop(self, inputs):
        for l in self.layers:
            inputs = l.present(inputs)
            
        return inputs[0]
            
    def backprop(self, prediction, target):
        for i in range(len(self.layers)-1, -1, -1):
            layer = self.layers[i]
            layer.backprop(prediction, target, i==(len(self.layers) - 1))
    
    
def MSE(network, data, targets):
    expected = np.array(list(map(lambda x: network.fowardprop(x), data)))
#     print(expected)
    return (1.0/2.0) * np.sum(np.power(np.subtract(expected, targets), 2))

def gradientHiddenLayer(neuron, neuronNumber, folowingLayer):
    numWeights = len(neuron.getWeights())
    grad = np.zeros(numWeights)
    
    # Compute the delta of current node
    dE_dy = 0
    for i in range(0, len(folowingLayer.getLayer())):
        fn = folowingLayer.getLayer()[i]
        delta = fn.getDelta()
        w = deltaW(neuronNumber, fn)
#         print(w)
        
        dE_dy += delta * w
        
    
#     print(dE_dy)
    
    for i in range(0, numWeights):
        grad[i] = -dE_dy * deltaW(i, neuron)
        
        
    neuron.setDelta(-dE_dy)
    
    return grad
    
    
def gradientOutputLayer(neuron, target, prediction):
    numWeights = len(neuron.getWeights())
    grad = np.zeros(numWeights)
    
    dE_dy = -(target - prediction)
    
    for i in range(0, numWeights):
        grad[i] = -dE_dy * deltaW(i, neuron)
    
    neuron.setDelta(-dE_dy)
#     print(grad)
    return grad

def UNITFUNC(x):
    if x >= 0:
        return 1
    
    return 0

# Compute W_{a,b}
def deltaW(i, neuron):
    muBs = np.delete(neuron.getMus(), i)
    t = UNITFUNC(np.min(muBs) - neuron.getMus()[i]) * UNITFUNC(neuron.getWeights()[i] - neuron.getInput()[i])
    return t

In [17]:
def trainNANDNetwork(data, targets, inputNodes, hLayers, outNodes):
    network = NANDNetwork(inputNodes, hLayers, outNodes)
    print(network)
    print("Initial Loss: ", MSE(network, data, targets))
    pterb = 0.00001
    
    for i in range(1, 10000):
        
#         if (MSE(network, data, targets) < 0.0000000000001):
#             break
        if i%1000 == 0:
            print()
            print("Iteration -> " + str(i))
            print("Loss: ", MSE(network, data, targets))
            
        for i in range(0, len(network.getLayers())):
            layer = network.getLayers()[i]
            for j in range(0, len(layer.getLayer())):
                neuron = layer.getLayer()[j]
                grad = np.zeros(len(neuron.getWeights()))
            
                for k in range(0, len(neuron.getWeights())):
                    g = np.zeros(len(neuron.getWeights()))
                    g[k] = -pterb

                    oldSSE = MSE(network, data, targets)
                    neuron.updateWeights(g)
                    newSSE = MSE(network, data, targets)
                    neuron.updateWeights(-g)
                
                    grad[k] = (newSSE - oldSSE)/pterb
                
#             print(grad)
                neuron.updateWeights(grad * 0.02)    
        
#         for j in range(0, len(data)):
#             prediction = network.fowardprop(data[j])
#             network.backprop(prediction, targets[j])
            
    print("Trained Loss: ", MSE(network, data, targets))
    return network

def checkGrad(pterb, threshold, inputNodes, hLayers, outNodes):
    network = NANDNetwork(inputNodes, hLayers, outNodes)
    
    print("Computing Numerical Grads")
    for i in range(0, len(network.getLayers())):
        layer = network.getLayers()[i]
        for j in range(0, len(layer.getLayer())):
            neuron = layer.getLayer()[j]
            grad = np.zeros(len(neuron.getWeights()))
            
            for k in range(0, len(neuron.getWeights())):
                g = np.zeros(len(neuron.getWeights()))
                g[k] = -pterb
                
                oldSSE = MSE(network, data, targets)
                neuron.updateWeights(g)
                newSSE = MSE(network, data, targets)
                neuron.updateWeights(-g)
                
                grad[k] = (newSSE - oldSSE)/pterb
                
#             print(grad)
            neuron.setCheckGrad(grad)
    
    print("Running Back Prop")
    for j in range(0, len(data)):
        prediction = network.fowardprop(data[j])
        network.backprop(prediction, targets[j])
        
        
    print("Checking Grad")
    for i in range(0, len(network.getLayers())):
        layer = network.getLayers()[i]
        for j in range(0, len(layer.getLayer())):
            neuron = layer.getLayer()[j]
            
            diff = np.absolute(neuron.getCheckGrad() - neuron.getGrad())
            for k in diff:
                if k > threshold:
                    print("GRAD WRONG[ " + str(i) + "," + str(j) + " ]: Got " + str(neuron.getGrad()) + " Should be " + str(neuron.getCheckGrad()))
                    break
            
            

In [18]:
# NAND Gate Data
# data = np.array([[-1.0,-1.0],[1.0,-1.0],[-1.0,1.0],[1.0,1.0]])
# targets = np.array([1.0,1.0,1.0,-1.0])



# NOT Gate Data
# data = np.array([[-1.0, -1.0, -1.0], [1.0, -1.0, -1.0]])
# targets = np.array([1.0, -1.0])
# trainNANDNetwork(data, targets, 3, [], 1)


# AND Gate Data
# data = np.array([[-1.0, -1.0, 1.0], [-1.0, 1.0, 1.0], [1.0, -1.0, 1.0], [1.0, 1.0, 1.0]])
# targets = np.array([-1.0, -1.0, -1.0, 1.0])

# OR Gate Data 
data = np.array([[-1.0, -1.0], [-1.0, 1.0], [1.0, -1.0], [1.0, 1.0]])
targets = np.array([-1.0, 1.0, 1.0, 1.0])

# data = np.array([[-1.0, -1.0], [-1.0, 1.0], [1.0, -1.0], [1.0, 1.0]])
# targets = np.array([1.0, 1.0, -1.0, 1.0])

# checkGrad(0.0001, 0.0001, 2, [2], 1)
trainNANDNetwork(data, targets, 2, [2], 1)

Layer 1 -> [ 0.  0.] ,[ 0.  0.] ,
Layer 2 -> [ 0.  0.] ,

Initial Loss:  2.0

Iteration -> 1000
Loss:  2.0

Iteration -> 2000
Loss:  2.0

Iteration -> 3000
Loss:  2.0


KeyboardInterrupt: 